In [4]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [5]:
pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [6]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = './data'

In [7]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

Объединим файлы в один датасет.

In [8]:
data = pd.concat([pd.read_json(file, lines=True, encoding = 'utf-8') for file in files][:1], axis=0, ignore_index=True)

In [9]:
data.shape

(999, 5)

Каждой статье приписано какое-то количество ключевых слов. Допустим, что это единственно правильный набор ключевых слов (что конечно не так, но других данных у нас нет). Наша задача - придумать как извлекать точно такой же список автоматически.  
Зададим несколько метрик, по которым будем определять качество извлекаемых ключевых слов - точность, полноту, ф1-меру и меру жаккарда.

In [11]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Проверим, что всё работает как надо.

In [12]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# Тупое решение.

Давайте не будем думать, а попробуем сразу придумать какое-то решение.

Возьмем первые 5 слов из заголовка.

In [13]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:5]))

Precision -  0.06
Recall -  0.06
F1 -  0.06
Jaccard -  0.03


Или 10.

In [14]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:10]))

Precision -  0.06
Recall -  0.07
F1 -  0.06
Jaccard -  0.03


Теперь попробуем взять самые частотные слова.

In [15]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 [x[0] for x in Counter(x.lower().split()).most_common(10)]))

Precision -  0.02
Recall -  0.04
F1 -  0.02
Jaccard -  0.01


Или вообще рандомные слова.

In [16]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 np.random.choice(list(set(x.lower().split())), 10)))

Precision -  0.01
Recall -  0.01
F1 -  0.01
Jaccard -  0.0


Теперь давайте посмотрим, что вообще извлекается.

In [17]:
data['title'].apply(lambda x: x.lower().split()[:10]).head(10)

0                         ["молодежное, "яблоко":, оппозиционная, деятельность, становится, опасной]
1                                                                 ["газпрома", на, всех, не, хватит]
2                                                   [бесконечная, партия, в, четырехмерные, шахматы]
3    [экс-депутат,, осужденная, за, фальсификацию, выборов,, оказалась, членом, "боевого, братства"]
4                               [новая, москва, останется, территорией, экологической, безопасности]
5                                [f1., гран-при, сша, прошел, без, четырех, машин, и, со, «стопкой»]
6                                          [100, ведущих, политиков, россии, в, феврале, 2018, года]
7                                               [закон, "о, культуре", принимают, на, фоне, арестов]
8                                    [насколько, реальна, газовая, подоплека, сирийского, конфликта]
9                                  [фсб:, в, калужской, области, задержаны, четверо, участн

In [18]:
data['content'].apply(lambda x: [x[0] for x in Counter(x.lower().split()).most_common(10)]).head(10)

0                                                      [в, и, на, не, что, –, его, «молодежное, с, это]
1                                                            [в, и, на, –, млрд., куб., по, к, газа, м]
2                                                                 [в, –, и, не, я, но, что, это, на, с]
3                                                       [в, на, и, ким, по, –, что, видео, он, зинаиды]
4                                              [в, и, на, новой, площадью, москвы, –, развития, с, для]
5                                                             [в, на, и, не, с, но, уже, что, у, гонки]
6                                                  [на, в, (с, место)., и, рф, позиции, влияние, по, с]
7                                                        [в, и, –, по, с, культуре, не, из, будет, как]
8                                                              [в, и, на, с, что, для, по, –, не, газа]
9    [в, рф, террористической, организации, задержаны, –, четвер

Когда извлекаются частотные слова, то список почти полностью состоит из всяких стоп-слов. Также из-за плохой токенизации некоторые слова в обоих списках - пунктуация или слова с пунктуацией на концах. К тому же извлекаемые слова ненормализованы, а правильные ключевые слова - наоборот.

## Токенизация, удаление стоп-слов и нормализация.

In [436]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = open('stopw.txt', 'r', encoding = 'utf-8').read()

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [437]:
data['content_norm'] = data['content'].apply(normalize)

In [438]:
data['title_norm'] = data['title'].apply(normalize)

In [439]:
data['title_norm'].head(10)

0            [молодёжный, яблоко, оппозиционный, деятельность, становиться, опасный]
1                                                                 [газпром, хватить]
2                                      [бесконечный, партия, четырехмерный, шахматы]
3    [экс-депутат, осудить, фальсификация, выбор, оказаться, член, боевой, братство]
4                 [новый, москва, остаться, территория, экологический, безопасность]
5                    [f1, гран-при, сша, пройти, без, четыре, машина, стопка, штраф]
6                                [100, ведущий, политик, россия, февраль, 2018, год]
7                                           [закон, культура, принимать, фон, арест]
8                     [насколько, реальный, газовый, подоплёка, сирийский, конфликт]
9                       [фсб, калужский, область, задержать, четверо, участник, иго]
Name: title_norm, dtype: object

Попробуем те же самые методы.

In [440]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.12
Recall -  0.23
F1 -  0.15
Jaccard -  0.08


In [441]:
evaluate(data['keywords'],data['title_norm'].apply(lambda x: x[:10]))

Precision -  0.13
Recall -  0.14
F1 -  0.12
Jaccard -  0.07


Качество сильно улучшилось! Можно теперь ещё раз посмотреть, что плохого извлекается.

In [442]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0                [яблоко, молодёжный, акция, год, активист, деятельность, политика, московский, власть, задача]
1                             [миллиард, газа, год, куб, газпром, добыча, 2020, должный, производитель, страна]
2                                   [год, книга, роман, писать, выйти, мир, перевод, век, стихотворение, можно]
3                                 [зинаида, видео, год, журналист, суд, дело, бывший, футиный, рубль, судебный]
4                           [площадь, территория, новый, москва, га, столица, тинао, парковый, развитие, парка]
5                                     [гонка, два, команда, место, позиция, из-за, круг, чемпионат, три, пилот]
6                        [место, влияние, рф, позиция, глава, россия, президент, сергей, политический, рейтинг]
7                  [культура, закон, сфера, стд, разработать, концепция, проект, изменение, сообщество, услуга]
8                                [газопровод, сирия, год, турция, газа, россия, европа, катар, турецкий,

Ещё остались некоторые стоп-слова. Вместо того, чтобы расширять список, давайте попробуем выкинуть несуществительные.

In [443]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [444]:
data['content_norm'] = data['content'].apply(normalize)

In [445]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


Ещу улучшения!

In [446]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(10)

0        [яблоко, акция, год, активист, деятельность, политика, власть, задача, молодая, человек]
1            [миллиард, газа, год, куб, газпром, добыча, производитель, страна, прогноз, холдинг]
2                [год, книга, роман, мир, перевод, стихотворение, читатель, жанр, поэзия, работа]
3              [зинаида, видео, год, журналист, суд, дело, рубль, процесс, заседание, экспертиза]
4                   [площадь, территория, москва, га, столица, тинао, развитие, парка, парк, год]
5                    [гонка, команда, место, позиция, круг, чемпионат, пилот, бокс, заезд, льюис]
6               [место, влияние, рф, позиция, глава, россия, президент, сергей, рейтинг, участие]
7    [культура, закон, сфера, концепция, проект, изменение, сообщество, услуга, учреждение, дело]
8                    [газопровод, сирия, год, турция, газа, россия, европа, катар, поток, проект]
9                 [участник, рф, организация, государство, область, центр, связь, фсб, март, год]
Name: content_norm, 

Не очень значимые слова все ещё остались. Давайте попробуем отсеять стоп-слова с помощью tfidf.

Воспользуемся TfidfVectorizer.

In [448]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [449]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,1), min_df=3, norm = 'l1', max_df = 0.8)

In [450]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=None, min_df=3,
        ngram_range=(1, 1), norm='l1', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [451]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [452]:
texts_vectors = tfidf.transform(data['content_norm_str'])

In [453]:
len(id2word)

8037

Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [454]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

word2idf - нужны индексы для rutermextractor и других попыток побить бейзлайн

In [455]:
word2idf_ = {i:j for i,j in list(zip(list(id2word.values()),list(tfidf.idf_))) }

In [456]:
list(word2idf_.items())[:10]

[('10', 5.509860006183766),
 ('100', 5.199705077879927),
 ('11', 5.605170185988092),
 ('115', 6.521460917862246),
 ('12', 6.115995809754082),
 ('120', 6.521460917862246),
 ('13', 5.605170185988092),
 ('14', 6.115995809754082),
 ('15', 5.710530701645918),
 ('150', 6.115995809754082)]

In [458]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


Результат ещё немного улучшился. Немного подросла точность. Теперь вместо стоп-слов в ключевые попадают имена и все такое. Иногда это хорощо, а иногда нет (собянин - может быть ключевым словом, а дарья - вряд ли)

Возьмем этот результат за baseline. 

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09

## Попробуем графы!

Большая часть методов для извлечения ключевых слов основана на применении графов. Основная идея - каким-то образом перевести текст в граф, а затем каким-то образом расчитать важность каждого узла и вывести топ-N самых важных узлов.  

Перевод текста в граф -  не тривиальная задача. Часто применяют такой подход - построим матрицу совстречаемости слов (в каком-то окне), эта матрица будет нашей матрицей смежности.

Для выбора важных узлов часто используют простой randow walk. Алгоритм примерно такой:  
1) Каким-то образом выбирается первый узел графа (например, случайно из равномерного распределения)  
2) на основе связей этого узла с другими, выбирается следующий узел  
3) шаг два повторяется некоторое количество раз (например, тысячу) __*чтобы не зацикливаться, с какой-то вероятностью мы случайно перескакиваем на другой узел (даже если он никак не связан с текущим, как в шаге 1)__  
5) на каждом шаге мы сохраняем узел в котором находимся  
6) в конце мы считаем в каких узлах мы были чаще всего и выводим top-N  


Предполагается, что мы часто будем приходить в важные узлы графа.

In [73]:
from itertools import combinations

Для наглядности реализуем этот подход без networkx. 

In [74]:
def get_kws(text, top=5, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        m[i] /= np.sum(m[i])
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        if go_random:
            n = np.random.choice(len(vocab))
        
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    next_n = np.random.choice(range(rang), p=matrix[n])
    return next_n
    


In [75]:
%%time
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 10, 10))

Wall time: 42.3 s


In [76]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.11
Recall -  0.21
F1 -  0.14
Jaccard -  0.08


In [77]:
keywords_rw.head(10)

0         [яблоко, деятельность, год, активист, выборы, женщина, борьба, акция, право, рамка]
1             [газа, миллиард, газпром, год, метр, куб, население, внедрение, добыча, страна]
2         [год, книга, россия, роман, жанр, москва, мениппея, стихотворение, читатель, герой]
3        [ким, зинаида, документ, владивосток, срок, год, приморье, адвокат, свобода, монтаж]
4           [год, территория, площадь, москва, парк, га, земля, столица, тинао, пространство]
5                  [гонка, позиция, команда, бокс, место, чемпионат, этап, мир, машина, круг]
6          [место, рф, влияние, глава, президент, позиция, сергей, политолог, регион, россия]
7       [культура, закон, изменение, дело, акт, сфера, концепция, союз, первое, председатель]
8                [газопровод, катар, год, сирия, военный, россия, поток, газ, проект, турция]
9    [участник, организация, рф, сирия, исламист, округа, эмиссар, житель, прошедшее, ячейка]
Name: content_norm, dtype: object

Попбруем теперь важность считать с помощью какой-нибудь метрики из networkx.

In [78]:
import networkx as nx

In [79]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.pagerank(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [80]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 10))

Wall time: 1min 53s


In [81]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09


Результаты не превосходят tfidf, но и не сильно уступают. Явно можно что-то доработать и превзойти baseline.

Готовое решение есть в gensim. Давайте попробуем его.

In [83]:
from gensim.summarization import keywords

In [84]:
gensim_kws = data['content_norm'].apply(lambda x: keywords(' '.join(x)).split('\n')[:10])

In [85]:
evaluate(data['keywords'], gensim_kws)

Precision -  0.06
Recall -  0.11
F1 -  0.08
Jaccard -  0.04


## Эксперименты

## rutermextract

In [382]:
def normalize_tag(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS in ['NOUN',"ADJF", 'ADJS', 'VERB', 'INFN', 'PRTF', 'PRTS']]

    return ' '.join(words)

def normalize_kw_tag(words):
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS in ['NOUN',"ADJF", 'ADJS', 'VERB', 'INFN', 'PRTF', 'PRTS']]

    return words

def normalize_kw_orig(words):
    words = ' '.join(words).split(' ')
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words]

    return words

In [383]:
data['keywords_norm'] = data.keywords.apply(normalize_kw_orig)

In [413]:
from rutermextract import TermExtractor
term_extractor = TermExtractor()

def rutermextr(text):
    key_te = []
    for term in term_extractor(text, limit = 20, nested = True, weight=lambda term: word2idf_.get(term.normalized, 1.0) * term.count):
        key_te.append(term.normalized)
    #return list(set(' '.join(key_te).split(' ')))
    return key_te

data['keywords_te_new'] = data.content_norm_str.apply(rutermextr)

C:\ProgramData\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [414]:
data.keywords_te_new = data.keywords_te_new.apply(normalize_kw_tag)

In [415]:
evaluate(data.keywords_norm, data.keywords_te_new)

Precision -  0.14
Recall -  0.43
F1 -  0.2
Jaccard -  0.11


In [416]:
data['content_nt'] = data.content.apply(normalize_tag)

In [417]:
tfidf.fit(data['content_nt'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [418]:
data['keywords_te_new'] = data.content_nt.apply(rutermextr)
data['keywords_nt'] = data.keywords_te_new.apply(normalize_kw_tag)

C:\ProgramData\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [419]:
evaluate(data.keywords_norm, data.keywords_nt)

Precision -  0.14
Recall -  0.43
F1 -  0.2
Jaccard -  0.11


## Rake

In [421]:
from rake_nltk import Metric, Rake

r = Rake(language = 'russian', min_length = 1, max_length = 1)

def rake_key(text):
    key_words = []
    r.extract_keywords_from_text(text)
    return r.get_ranked_phrases()

data['keywords_rake'] = data.content_norm_str.apply(rake_key)

In [423]:
evaluate(data.keywords_norm, data.keywords_rake)

Precision -  0.01
Recall -  0.0
F1 -  0.0
Jaccard -  0.0


## Yake

In [406]:
import yake

In [407]:
def yake_key(text):
    custom_kwextractor = yake.KeywordExtractor(lan="ru", n = 1, dedupLim=0.8, windowsSize=2, top=20)
    keywords = custom_kwextractor.extract_keywords(text)
    #custom_kwextractor = yake.KeywordExtractor(lan="ru", n = 2, windowsSize=3, top=10)
    #keywords_2 = custom_kwextractor.extract_keywords(text)
    #keywords = keywords_1 + keywords_2
    return [i[1] for i in keywords]
    
data['keywords_yake'] = data.content_norm_str.apply(yake_key)

In [409]:
evaluate(data.keywords_norm, data.keywords_yake)

Precision -  0.14
Recall -  0.45
F1 -  0.2
Jaccard -  0.12


In [410]:
data['keywords_yake_pos'] = data.keywords_yake.apply(normalize_kw_tag)

In [424]:
evaluate(data.keywords_norm, data.keywords_yake_pos)

Precision -  0.09
Recall -  0.56
F1 -  0.16
Jaccard -  0.09


## Topic Rank

In [470]:
!pip install summa

  Stored in directory: C:\Users\estiu\AppData\Local\pip\Cache\wheels\6a\09\68\e2f2861c01d86407c3fa5220826ed7eed2abaa56b001be5970
Successfully built summa


In [501]:
from summa import keywords

def textrank_kw(text):
    kw = []
    kw = keywords.keywords(text, words = 10).split('\n')
    kw = ' '.join(kw).split(' ')
    return kw

data['keywords_tr'] = data.content_nt.apply(textrank_kw)

In [502]:
data['keywords_tr_pos'] = data.keywords_tr.apply(normalize_kw_tag)

In [503]:
data.head()

,content,keywords,summary,title,url,content_norm,title_norm,content_norm_str,keywords_te_new,content_nt,keywords_nt,keywords_rake,keywords_yake,keywords_yake_pos,keywords_norm,keywords_tr,keywords_tr_pos
0,"Многие интересуются, зачем нужна «Яблоку» молодежная фракция? Основной задачей «Молодежного «Яблока» является привлечение молодых людей к участию в выборах и деятельности партии. «Молодежное «Яблоко» работает более чем в 10 регионах. Единого руководства у нас нет, но мы стараемся координировать свою деятельность и периодически проводим акции на федеральном уровне.\nМы ведем борьбу с обязательным воинским призывом. Военный – это профессия, а не обязанность. Молодые люди вправе сами распоряжаться своей жизнью и не терять целый год, отдавая государству «долг», который они у него не занимали. По мнению одного из ведущих специалистов в области оборонной политики Алексея Арбатова, переход на контрактную армию будет стоить лишь 2% военного бюджета.\nТакже на федеральном уровне «Молодежное «Яблоко» проводило акции за освобождение политзаключенных и против вмешательства России во внутреннюю политику Украины.\nРасскажу о московских активистах. Виктору Петрунину – 19 лет, он пришел к нам боль...","[яблоко, молодежь, молодежное яблоко]",,"""Молодежное ""Яблоко"": оппозиционная деятельность становится опасной",http://www.ng.ru/ng_politics/2017-04-18/11_6976_apple.html,"[яблоко, фракция, задача, яблоко, привлечение, молодая, человек, участие, выборы, деятельность, партия, яблоко, регион, руководство, деятельность, акция, уровень, борьба, призыв, военный, профессия, обязанность, молодая, человек, жизнь, год, государство, долг, мнение, специалист, область, политика, алексей, арбатов, переход, армия, военный, бюджет, уровень, яблоко, акция, освобождение, политзаключённый, вмешательство, россия, политика, украина, активист, виктор, петрунин, год, год, москва, семья, взгляд, виктор, детство, политика, виктор, власть, крым, война, украина, убийство, борис, немцов, сторона, школа, сила, смена, власть, россия, дарья, новичкова, влияние, преподаватель, обществознание, ситуация, страна, искренность, заинтересованность, вопрос, дарья, материал, сми, время, наблюдение, позиция, образ, команда, единомышленник, пример, ровесник, дарья, яблоко, яблоко, клуб, рамка, лекция, кинопоказ, ...]","[молодёжный, яблоко, оппозиционный, деятельность, становиться, опасный]",яблоко фракция задача яблоко привлечение молодая человек участие выборы деятельность партия яблоко регион руководство деятельность акция уровень борьба призыв военный профессия обязанность молодая человек жизнь год государство долг мнение специалист область политика алексей арбатов переход армия военный бюджет уровень яблоко акция освобождение политзаключённый вмешательство россия политика украина активист виктор петрунин год год москва семья взгляд виктор детство политика виктор власть крым война украина убийство борис немцов сторона школа сила смена власть россия дарья новичкова влияние преподаватель обществознание ситуация страна искренность заинтересованность вопрос дарья материал сми время наблюдение позиция образ команда единомышленник пример ровесник дарья яблоко яблоко клуб рамка лекция кинопоказ тема наступление консерватизм обсуждение альтернатива ситуация экономика участник стол мероприятие место дискуссия участие люба гость яблоко повестка ноябрь прошлое год карелия акц...,"[яблоко, активист, акция, дарья, деятельность, молодая, политика, виктор, выборы, силовик, репрессия, тимур, молодёжный яблоко, год, насилие, убийство, освобождение, отделение, власть, молодёжь]",интересоваться нужный яблоко молодёжный фракция основной задача молодёжный яблоко являться привлечение молодая человек участие выборы деятельность партия молодёжный яблоко работать регион единый руководство стараться координировать деятельность проводить акция федеральный уровень вести борьба обязательный воинский призыв военный профессия обязанность молодая человек сам распоряжаться свой жизнь терять целый год государст

In [504]:
evaluate(data.keywords_norm, data.keywords_tr_pos)

Precision -  0.2
Recall -  0.3
F1 -  0.23
Jaccard -  0.14


В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://goo.gl/forms/H9lBH9wCxqq1T0ru2

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другое.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 1 балл).